In [21]:
#Necessary imports for plotting, pandas, math operations, and multiple print statements.
import pandas as pd
from scipy.stats import chi2, chi2_contingency
from IPython.display import Image
import matplotlib.pyplot as plt
from math import pi
import numpy as np
import seaborn as sns
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [22]:
def removeDuplicates(scripts_basics_df):
    # Key = IMDB ID , Value = Size of script
    smallestMovieScriptMap = {}
    # Key = IMDB ID , Value = index
    keepIndexMap = {}

    #show all columns
    scripts_basics_df.dtypes

    scripts_basics_df['Scripts'] = scripts_basics_df['Scripts'].apply(str)

    for index, row in scripts_basics_df.iterrows():
        constID = row['IMDB ID']
        currentSize = len(row['Scripts'])

        if constID in smallestMovieScriptMap:
            smallestSize = smallestMovieScriptMap[constID]
            if currentSize < smallestSize:
                smallestMovieScriptMap[constID] = currentSize
                keepIndexMap[constID] = index
        else:
            smallestMovieScriptMap[constID] = currentSize
            keepIndexMap[constID] = index
    # keep this removed, this removes first instance
    # scripts_basics_df = scripts_basics_df[~scripts_basics_df.index.duplicated(keep='first')]
    return scripts_basics_df[scripts_basics_df.index.isin(keepIndexMap.values())]


In [23]:
#Read basics.tsv
basics_df = pd.read_csv("datasets/basics.tsv", sep="\t")

#Filter basics_df
basics_df = basics_df[basics_df["titleType"] == "movie"]
basics_df = basics_df[basics_df["genres"] != "\\N"]
# basics_df = basics_df[basics_df["startYear"] != "\\N"]
# basics_df["startYear"] = basics_df["startYear"].apply(int)
# basics_df = basics_df[basics_df["startYear"] >= 1960]
basics_df = basics_df.set_index("tconst")

#Filter akas_df
akas_df = pd.read_csv("datasets/akas.tsv", sep="\t", dtype={
    "titleId": "string", 
    "ordering": int, 
    "title": "string", 
    "region":"string",
    "language":"string"
})

akas_df = akas_df[akas_df["region"].isin(["US","CA"])]
akas_df = akas_df[akas_df["language"] == "en"]
akas_df = akas_df.set_index("titleId")

#Join
akas_basics_df = basics_df.join(akas_df, how='inner')

#Read scripts.csv
scripts_df = pd.read_csv("datasets/scripts.csv", dtype={
    "IMDB ID": "string", 
    "Scripts": "string", 
    "release year": int, 
});
scripts_df = scripts_df.set_index("IMDB ID");
scripts_df = scripts_df.drop(['genres'], axis=1)

scripts_basics_df = akas_basics_df.join(scripts_df, how='inner')
scripts_basics_df=scripts_basics_df.reset_index()
scripts_basics_df = scripts_basics_df.rename(columns={'index': 'IMDB ID'})
scripts_basics_df = removeDuplicates(scripts_basics_df)


#only show original title and scripts
scripts_basics_df[["originalTitle", "Scripts"]]

C:\Users\Kotori\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Kotori\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,originalTitle,Scripts
0,Kate & Leopold,"Time. Time, it has been proposed is the fourth..."
2,Zandalee,<NA>
3,Backdraft,<font color=orange>© <font color=orange>© P <f...
5,Barton Fink,Kissing it all good-bye. These four stinkin' w...
6,Bill & Ted's Bogus Journey,[SUSPENSEFUL MUSIC PLAYING] It is time. They'v...
...,...,...
3124,Hickok,"No, no, you keep your goddamn head down, all r..."
3126,Lost in London,The Venom - $5 million GTD Poker Tourney Downl...
3127,A Bad Moms Christmas,♪ I'll have a blue Christmas ♪ ♪ Without you ♪...
3128,SPF-18,real life and the movies can get a little mixe...
